In [1]:
import gymnasium as gym
import torch
import torch.nn as nn
from skrl.envs.wrappers.torch import wrap_env
from skrl.memories.torch import RandomMemory
from skrl.models.torch import DeterministicMixin, Model, MultiCategoricalMixin
from skrl.trainers.torch import ParallelTrainer
from skrl.utils import set_seed
from torch.nn import TransformerEncoder, TransformerEncoderLayer

from preprocess import preprocess
from skrl.utils.spaces.torch import unflatten_tensorized_space

set_seed(42)

gym.register(
    id="MultiDatasetDiscretedTradingEnv",
    entry_point="only_features_env:MultiDatasetDiscretedTradingEnv",
    disable_env_checker=True,
)

[skrl:INFO] Seed: 42


In [2]:
env_cfg = dict(
    id="MultiDatasetDiscretedTradingEnv",
    dataset_dir="./data/train/month_15m/**/**/*.pkl",
    preprocess=preprocess,
    positions=[-1, 1],
    multiplier=[1, 2, 5],
    trading_fees=0.01,
    borrow_interest_rate=0.03,
    portfolio_initial_value=1000,
    max_episode_duration="max",
    verbose=1,
    window_size=60,
)

In [3]:
env = gym.make(**env_cfg)
obs = env.observation_space
env = wrap_env(env, wrapper="gymnasium")

[skrl:INFO] Environment wrapper: gymnasium
[skrl:WARNING] Failed to check for a vectorized environment: module 'gymnasium.experimental' has no attribute 'vector'


In [4]:
device = env.device
replay_buffer_size = 1024 * 8 * env.num_envs
memory_size = int(replay_buffer_size / env.num_envs)
memory = RandomMemory(memory_size=memory_size, num_envs=env.num_envs, device=device, replacement=False)

In [5]:
class LearnablePositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        self.position_embedding = nn.Embedding(max_len, d_model)

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        positions = torch.arange(0, seq_len, device=x.device).unsqueeze(0)
        positions = positions.expand(batch_size, seq_len)
        position_encoded = self.position_embedding(positions)
        return x + position_encoded


class SharedNoFC(MultiCategoricalMixin, DeterministicMixin, Model):
    def __init__(
        self,
        observation_space,
        action_space,
        device,
        clip_actions=False,
        unnormalized_log_prob=True,
        reduction="sum",
    ):
        Model.__init__(self, observation_space, action_space, device)
        MultiCategoricalMixin.__init__(self, unnormalized_log_prob, reduction)
        DeterministicMixin.__init__(self, clip_actions)
        
        self._shared_features = None
        self.net_projection = nn.Sequential(
            nn.Conv1d(7, 8, kernel_size=1, padding=1),
        )

        # Transformer Encoder for self-attention
        transformer_layer = TransformerEncoderLayer(
            d_model=8,  # The size of the input feature vector
            nhead=4,      # Number of attention heads
            dim_feedforward=256,  # The size of the feedforward network in the encoder
            dropout=0.1,
            batch_first=True  # Use batch_first for better inference performance
        )
        self.positional_encoding = LearnablePositionalEncoding(d_model=8)
        self.transformer_encoder = TransformerEncoder(transformer_layer, num_layers=2)

        # Output heads using Conv1d with additional layers to capture interactions
        self.policy_head = nn.Sequential(
            # nn.Conv1d(8, 16, kernel_size=3, padding=1),
            # nn.ReLU(),
            nn.Conv1d(8, self.num_actions, kernel_size=1), # 7 = seq_length
            nn.AdaptiveAvgPool1d(1)  # 마지막에 시퀀스 길이를 1로 줄임
        )
        self.value_head = nn.Sequential(
            # nn.Conv1d(8, 16, kernel_size=3, padding=1),
            # nn.ReLU(),
            nn.Conv1d(8, 1, kernel_size=1),
            nn.AdaptiveAvgPool1d(1)
        )

    def act(self, inputs, role):
        if role == "policy":
            return MultiCategoricalMixin.act(self, inputs, role)
        elif role == "value":
            return DeterministicMixin.act(self, inputs, role)

    def compute(self, inputs, role):
        states = unflatten_tensorized_space(obs, inputs["states"])

        if role == "policy":
            features = states.permute(0, 2, 1)
            features = self.net_projection(features)
            features = self.positional_encoding(features.permute(0, 2, 1)) # torch.Size([32, 7, 128])
            features = features + self.transformer_encoder(features) # torch.Size([32, 7, 128])

            self._shared_features = features.permute(0, 2, 1) # torch.Size([32, 128, 7])
            actions = self.policy_head(self._shared_features) # torch.Size([32, 5, 1])
            # print(actions)
            return actions.squeeze(-1), {} # torch.Size([32, 5])
            
        elif role == "value":
            if self._shared_features is None:
                features = states.permute(0, 2, 1)
                features = self.net_projection(features)
                features = self.positional_encoding(features.permute(0, 2, 1))
                features = features + self.transformer_encoder(features)

                shared_features = features.permute(0, 2, 1)
            else:
                shared_features = self._shared_features

            self._shared_output = None

            value = self.value_head(shared_features)
            return value.squeeze(-1), {}

In [6]:
models = {}
models["policy"] = SharedNoFC(env.observation_space, env.action_space, device)
models["value"] = models["policy"]

for model in models.values():
    model.init_parameters(method_name="normal_", mean=0.0, std=0.1)

In [7]:
from skrl.agents.torch.ppo import PPO_DEFAULT_CONFIG
from skrl.resources.schedulers.torch import KLAdaptiveLR

cfg = PPO_DEFAULT_CONFIG.copy()
cfg["rollouts"] = memory_size
cfg["learning_epochs"] = 32
cfg["mini_batches"] = 16
cfg["discount_factor"] = 0.99
cfg["learning_rate"] = 5e-4
cfg["learning_rate_scheduler"] = KLAdaptiveLR
cfg["learning_rate_scheduler_kwargs"] = {"kl_threshold": 0.01, "min_lr": 1e-7}

cfg["experiment"]["write_interval"] = 5000
cfg["experiment"]["checkpoint_interval"] = 100000
cfg["experiment"]["directory"] = "runs/torch/mddt"

In [8]:
import warnings
from skrl.agents.torch.ppo import PPO

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

agent = PPO(
    models=models,
    memory=memory,
    cfg=cfg,
    observation_space=env.observation_space,
    action_space=env.action_space,
    device=device,
)
path = "24-12-04_17-39-55-948072_PPO"
agent.load(f"/home/pitin/Desktop/hp/runs/torch/mddt/{path}/checkpoints/best_agent.pt")
cfg_trainer = {"timesteps": 10000000, "headless": True, "environment_info": "pc_counter"}
trainer = ParallelTrainer(cfg=cfg_trainer, env=env, agents=[agent])

In [9]:
states, infos = env.reset()
timestep = 0
timesteps = 3000
terminated = torch.tensor([[False] * env.num_envs])

# while (not terminated.any()):
for _ in range(1000):
    agent.pre_interaction(timestep=timestep, timesteps=timesteps)

    with torch.no_grad():
        actions = agent.act(states, timestep=timestep, timesteps=timesteps)[0]
        next_states, rewards, terminated, truncated, infos = env.step(actions)
        env.render()

    super(type(agent), agent).post_interaction(timestep=timestep, timesteps=timesteps)

    if env.num_envs > 1:
        states = next_states
    else:
        if terminated.any() or truncated.any():
            with torch.no_grad():
                states, infos = env.reset()
        else:
            states = next_states

env.save_for_render()

KeyError: 61

In [ ]:
from gym_trading_env.renderer import Renderer
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()

 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [05/Dec/2024 15:19:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 15:19:47] "GET /update_data/binance-BTCUSDT-15m.pkl_2024-12-04_17-38-26.pkl HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 15:19:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [05/Dec/2024 15:19:48] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 15:20:00] "GET /update_data/binance-SANDUSDT-15m.pkl_2024-12-05_08-25-24.pkl HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 15:20:00] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 15:20:59] "GET /update_data/binance-BTCUSDT-15m.pkl_2024-12-04_17-38-26.pkl HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2024 15:20:59] "GET /metrics HTTP/1.1" 200 -
